In [ ]:
import pandas as pd

# 读取原始数据文件
df1 = pd.read_csv(r'D:\songyue\tongtong\final_data.csv')
df1

In [ ]:
print(df1.columns)

In [ ]:
df1 = df1.drop_duplicates()

匹配性别

In [ ]:
print(df1.shape)

In [ ]:
import pandas as pd

# 读取原始数据文件
xingbie = pd.read_csv(r'D:\songyue\性别\result\result.csv', header=None)

# 添加列名
xingbie.columns = ["o_id", "姓名", "性别"]

# 显示数据框的前几行以验证
print(xingbie.head())

In [ ]:
# 依据唯一ID列，将df2中的US排名2023合并到frequency_distribution
merged_df_new2 = pd.merge(df1, xingbie[["o_id", "性别"]], on='o_id', how='left')

In [ ]:
merged_df_new2

In [ ]:
merged_df_new2 = merged_df_new2.drop_duplicates()

匹配专业

In [ ]:
# 读取原始数据文件
zhuanye = pd.read_csv('D:\songyue\专业清洗\待标注数据3_预测结果-mm.csv')
# 重命名唯一ID4为唯一ID
zhuanye = zhuanye.rename(columns={'数据': 'department_name'})
zhuanye= zhuanye.drop_duplicates()
# 依据唯一ID列，将df2中的US排名2023合并到frequency_distribution
merged_df_new3 = pd.merge(merged_df_new2, zhuanye[['department_name', '171类',"30类","11类","3类"]], on='department_name', how='left')

In [ ]:
print(merged_df_new3.shape)

In [ ]:
print(merged_df_new3.shape)

In [ ]:
merged_df_new3 = merged_df_new3.drop_duplicates()

In [ ]:
print(merged_df_new3.columns)

不区分样本：不同学历的'唯一ID'统计

In [ ]:
import pandas as pd

# 按照 'o_id' 和 'start_year_origin' 升序排列
merged_df_new3_sorted = merged_df_new3.sort_values(by=['o_id', 'start_year_origin'])
merged_df_new3_sorted

In [ ]:
# import pandas as pd

# # 按照 'o_id' 和 'start_year_origin' 升序排列
# merged_df_new3_sorted = merged_df_new3.sort_values(by=['o_id', 'start_year_origin'])

# 定义一个函数来获取样本并统计频率
def get_sample_and_count_frequency(df, filter_column, filter_value):
    # 筛选出特定条件的行
    sub_sample = df[df[filter_column] == filter_value]
    
    # 按 'o_id' 分组，取每组的最后一行
    last_rows = sub_sample.groupby('o_id').last().reset_index()
    
    # 统计 '唯一ID' 列的频率
    frequency_counts = last_rows['唯一ID'].value_counts().reset_index()
    frequency_counts.columns = ['唯一ID', '频率']
    
    return frequency_counts

# 统计 '本科统计' 为 1 的样本
undergraduate_frequency = get_sample_and_count_frequency(merged_df_new3_sorted, '本科统计', 1)

# 统计 '硕士统计' 为 1 的样本
master_frequency = get_sample_and_count_frequency(merged_df_new3_sorted, '硕士统计', 1)

# 统计 '博士统计' 为 1 的样本
doctoral_frequency = get_sample_and_count_frequency(merged_df_new3_sorted, '博士统计', 1)

# 输出结果
print("本科统计样本的唯一ID频率分布：")
print(undergraduate_frequency)

print("\n硕士统计样本的唯一ID频率分布：")
print(master_frequency)

print("\n博士统计样本的唯一ID频率分布：")
print(doctoral_frequency)


In [ ]:
# 保存结果到 Excel 文件
output_file_path = r"D:\songyue\tongtong\ORCID数据检查\不同学历——全球的机构分布\frequency_distribution.xlsx"

with pd.ExcelWriter(output_file_path) as writer:
    undergraduate_frequency.to_excel(writer, sheet_name='本科统计', index=False)
    master_frequency.to_excel(writer, sheet_name='硕士统计', index=False)
    doctoral_frequency.to_excel(writer, sheet_name='博士统计', index=False)

print(f"\n结果已保存到: {output_file_path}")

区分样本：分析有工作的样本——当前工作机构与本科院校层次的关系
筛选merged_df_new3_sorted 的'工作统计新'取值为1的行的"o_id”列唯一取值得到一个列表，然后从merged_df_new3_sorted 的'教育统计新'取值为1的行的"o_id”列唯一取值得到一个列表2，然后得到两个列表的交集的“o_id”的集合。然后从merged_df_new3_sorted取出相应的样本，可以做到吗

In [ ]:
import pandas as pd

# 假设 merged_df_new3_sorted 已经定义并按 'o_id' 和 'start_year_origin' 排序

# 筛选 '工作统计新' 列为 1 的行，并获取 'o_id' 的唯一值
o_id_work = merged_df_new3_sorted[merged_df_new3_sorted['工作统计新'] == 1]['o_id'].unique()

# 筛选 '教育统计新' 列为 1 的行，并获取 'o_id' 的唯一值
o_id_education = merged_df_new3_sorted[merged_df_new3_sorted['教育统计新'] == 1]['o_id'].unique()

# 计算两个列表的交集
common_o_ids = set(o_id_work).intersection(set(o_id_education))

# 从 merged_df_new3_sorted 中筛选出这些交集对应的样本
filtered_samples = merged_df_new3_sorted[merged_df_new3_sorted['o_id'].isin(common_o_ids)]

# 输出结果
print("符合条件的样本：")
print(filtered_samples)

# 可选：保存结果到 Excel
output_filtered_path = r"D:\songyue\tongtong\ORCID数据检查\符合条件的样本（有工作）.csv"
filtered_samples.to_csv(output_filtered_path, index=False)
print(f"\n结果已保存到: {output_filtered_path}")

In [ ]:
# 查看 filtered_samples 中 o_id 的个数
o_id_count = filtered_samples['o_id'].nunique()
print(f"符合条件的样本中 o_id 的个数为：{o_id_count}")

In [ ]:
print(filtered_samples.shape)
print(filtered_samples.columns)

In [ ]:
# import pandas as pd

# # 按照 'o_id' 和 'start_year_origin' 升序排列
# merged_df_new3_sorted = merged_df_new3.sort_values(by=['o_id', 'start_year_origin'])

# 定义一个函数来获取样本并统计频率
def get_sample_and_count_frequency(df, filter_column, filter_value):
    # 筛选出特定条件的行
    sub_sample = df[df[filter_column] == filter_value]
    
    # 按 'o_id' 分组，取每组的最后一行
    last_rows = sub_sample.groupby('o_id').last().reset_index()
    
    # 统计 '唯一ID' 列的频率
    frequency_counts = last_rows['唯一ID'].value_counts().reset_index()
    frequency_counts.columns = ['唯一ID', '频率']
    
    return frequency_counts

# 统计 '本科统计' 为 1 的样本
undergraduate_frequency = get_sample_and_count_frequency(filtered_samples, '本科统计', 1)

# 统计 '硕士统计' 为 1 的样本
master_frequency = get_sample_and_count_frequency(filtered_samples, '硕士统计', 1)

# 统计 '博士统计' 为 1 的样本
doctoral_frequency = get_sample_and_count_frequency(filtered_samples, '博士统计', 1)

# 输出结果
print("本科统计样本的唯一ID频率分布：")
print(undergraduate_frequency)

print("\n硕士统计样本的唯一ID频率分布：")
print(master_frequency)

print("\n博士统计样本的唯一ID频率分布：")
print(doctoral_frequency)
# 保存结果到 Excel 文件
output_file_path = r"D:\songyue\tongtong\ORCID数据检查\不同学历——全球的机构分布\有工作教育样本\frequency_distribution.xlsx"

with pd.ExcelWriter(output_file_path) as writer:
    undergraduate_frequency.to_excel(writer, sheet_name='本科统计', index=False)
    master_frequency.to_excel(writer, sheet_name='硕士统计', index=False)
    doctoral_frequency.to_excel(writer, sheet_name='博士统计', index=False)

print(f"\n结果已保存到: {output_file_path}")

In [ ]:
# 导出 filtered_samples 的前 1000 行到 Excel 文件
output_filtered_samples_path = r"D:\songyue\tongtong\ORCID数据检查\filtered_samples_top_1000.xlsx"

# 获取前 1000 行
top_1000_samples = filtered_samples.head(1000)

# 保存到 Excel
top_1000_samples.to_excel(output_filtered_samples_path, index=False)

print(f"\n前 1000 行已保存到: {output_filtered_samples_path}")


最高学历分析

In [ ]:
import pandas as pd

# 筛选 '教育统计新' 列为 1 的行，得到 edu 子数据
edu = filtered_samples[filtered_samples['教育统计新'] == 1]

# 获取所有 "o_id" 的唯一取值集合
list1 = set(edu['o_id'].unique())

# 列表2：获取最高学历为博士的 o_id 集合
list2 = set(edu[edu['博士统计'] == 1]['o_id'].unique())

# 列表3：从列表1中剔除列表2的 o_id 集合
list3 = list1 - list2

# 列表4：在列表3中获取“硕士统计”列取值为1的 o_id 集合
list4 = set(edu[(edu['o_id'].isin(list3)) & (edu['硕士统计'] == 1)]['o_id'].unique())

# 列表5：从列表3中剔除列表4的 o_id 集合
list5 = list3 - list4

# 列表6：在列表5中获取“本科统计”列取值为1的 o_id 集合
list6 = set(edu[(edu['o_id'].isin(list5)) & (edu['本科统计'] == 1)]['o_id'].unique())

# 计算各列表中 o_id 的数量
print(f"列表1中 o_id 个数：{len(list1)}")
print(f"列表2中 o_id 个数：{len(list2)}")
print(f"列表3中 o_id 个数：{len(list3)}")
print(f"列表4中 o_id 个数：{len(list4)}")
print(f"列表5中 o_id 个数：{len(list5)}")
print(f"列表6中 o_id 个数：{len(list6)}")

# 获取最高学历不详的 o_id 集合
unknown_education_o_ids = list1 - list2 - list4 - list6
print(f"最高学历不详的样本 o_id 列表：{unknown_education_o_ids}")

In [ ]:
import pandas as pd

# 创建一个字典以便于构建 DataFrame
data = {
    'o_id': [],
    '最高学历': []
}

# 添加博士样本
data['o_id'].extend(list2)
data['最高学历'].extend(['博士'] * len(list2))

# 添加硕士样本
data['o_id'].extend(list4)
data['最高学历'].extend(['硕士'] * len(list4))

# 添加疑似本科样本
data['o_id'].extend(list6)
data['最高学历'].extend(['本科'] * len(list6))

# 添加最高学历不详的样本
data['o_id'].extend(unknown_education_o_ids)
data['最高学历'].extend(['最高学历不详'] * len(unknown_education_o_ids))

# 创建 DataFrame
result_df = pd.DataFrame(data)

# 输出结果 DataFrame
print(result_df)

# 保存到 Excel 文件（可选）
output_path = r"D:\songyue\tongtong\ORCID数据检查\不同学历——全球的机构分布\有工作教育样本\最高学历统计.csv"
result_df.to_csv(output_path, index=False)

print(f"\n结果已保存到: {output_path}")


In [ ]:
# 进行频率统计
frequency_counts = result_df['最高学历'].value_counts().reset_index()
frequency_counts.columns = ['最高学历', '数量']

# 输出频率统计结果
print("学历频率统计：")
print(frequency_counts)

In [ ]:
import pandas as pd

# 假设 work_edu 已经定义

# 筛选条件：最高学历为博士，教育统计为1，并按“o_id”分组，获取“教育统计新”最后一行数据
edu_doc = (work_edu[(work_edu['最高学历'] == '博士') & (work_edu['教育统计'] == 1)]
           .groupby('o_id')
           .apply(lambda x: x.iloc[-1])
           [['o_id', '教育统计新']])

# 筛选条件：最高学历为博士，工作统计新为1，并按“o_id”分组，获取“工作统计新”最后一行数据
work_doc = (work_edu[(work_edu['最高学历'] == '博士') & (work_edu['工作统计新'] == 1)]
            .groupby('o_id')
            .apply(lambda x: x.iloc[-1])
            [['o_id', '工作统计新', 'type']])

# 输出 work_doc 的 type 列中各取值的频率
type_frequencies = work_doc['type'].value_counts()
print("work_doc 的 type 列各个取值的频率：\n", type_frequencies)


In [ ]:
import pandas as pd

# 假设 work_edu 已经定义

# 筛选条件：最高学历为博士，教育统计为1，并按“o_id”分组，获取“教育统计新”最后一行数据
edu_mas = (work_edu[(work_edu['最高学历'] == '博士') & (work_edu['教育统计'] == 1)]
           .groupby('o_id')
           .apply(lambda x: x.iloc[-1])
           [['o_id', '教育统计新']])

# 筛选条件：最高学历为博士，工作统计新为1，并按“o_id”分组，获取“工作统计新”最后一行数据
work_mas = (work_edu[(work_edu['最高学历'] == '博士') & (work_edu['工作统计新'] == 1)]
            .groupby('o_id')
            .apply(lambda x: x.iloc[-1])
            [['o_id', '工作统计新', 'type']])

# 输出 work_doc 的 type 列中各取值的频率
type_frequencies = work_mas['type'].value_counts()
print("work_doc 的 type 列各个取值的频率：\n", type_frequencies)

In [ ]:
import pandas as pd

# 假设 work_edu 已经定义

# 筛选条件：最高学历为博士，教育统计为1，并按“o_id”分组，获取“教育统计新”最后一行数据
edu_bac = (work_edu[(work_edu['最高学历'] == '博士') & (work_edu['教育统计'] == 1)]
           .groupby('o_id')
           .apply(lambda x: x.iloc[-1])
           [['o_id', '教育统计新']])

# 筛选条件：最高学历为博士，工作统计新为1，并按“o_id”分组，获取“工作统计新”最后一行数据
work_bac = (work_edu[(work_edu['最高学历'] == '博士') & (work_edu['工作统计新'] == 1)]
            .groupby('o_id')
            .apply(lambda x: x.iloc[-1])
            [['o_id', '工作统计新', 'type']])

# 输出 work_doc 的 type 列中各取值的频率
type_frequencies = work_bac['type'].value_counts()
print("work_doc 的 type 列各个取值的频率：\n", type_frequencies)

In [ ]:
import pandas as pd

# 假设 work_edu 已经定义

# 筛选条件：最高学历为博士，教育统计为1，并按“o_id”分组，获取“教育统计新”最后一行数据
edu_unsure = (work_edu[(work_edu['最高学历'] == '博士') & (work_edu['教育统计'] == 1)]
           .groupby('o_id')
           .apply(lambda x: x.iloc[-1])
           [['o_id', '教育统计新']])

# 筛选条件：最高学历为博士，工作统计新为1，并按“o_id”分组，获取“工作统计新”最后一行数据
work_unsure = (work_edu[(work_edu['最高学历'] == '博士') & (work_edu['工作统计新'] == 1)]
            .groupby('o_id')
            .apply(lambda x: x.iloc[-1])
            [['o_id', '工作统计新', 'type']])

# 输出 work_doc 的 type 列中各取值的频率
type_frequencies = work_unsure['type'].value_counts()
print("work_doc 的 type 列各个取值的频率：\n", type_frequencies)

合并数据

In [ ]:
import pandas as pd

# 假设 edu_doc、work_doc、edu_mas、work_mas、edu_bac、work_bac 已经定义

# 定义横向合并函数，包含后缀和学历类型标记
def merge_horizontal(edu_df, work_df, edu_suffix, work_suffix, degree_type):
    merged_df = pd.merge(
        edu_df.add_suffix(edu_suffix), 
        work_df.add_suffix(work_suffix), 
        left_index=True, 
        right_index=True, 
        how='outer'
    )
    merged_df['教育类型'] = degree_type
    return merged_df

# 逐个合并博士、硕士和本科的 edu 和 work 数据框
merged_doc = merge_horizontal(edu_doc, work_doc, '_edu', '_work', '博士')
merged_mas = merge_horizontal(edu_mas, work_mas, '_edu', '_work', '硕士')
merged_bac = merge_horizontal(edu_bac, work_bac, '_edu', '_work', '本科')

# 将三个合并后的数据框纵向合并
final_merged_df = pd.concat([merged_doc, merged_mas, merged_bac], ignore_index=True)

# 查看合并后的结果
print(final_merged_df)

In [ ]:
# 选择需要保留的列并重命名
final_merged_df = final_merged_df.rename(columns={
    '教育统计_edu': '教育统计',
    '工作统计新_edu': '工作统计新',
    '教育统计新_edu': '教育统计新',
    '性别_edu': '性别',
    '最高学历_edu': '最高学历',
    'o_id_edu': 'o_id'
})

# 删除多余列
final_merged_df = final_merged_df.drop(columns=[
    '教育统计_work', '工作统计新_work', '教育统计新_work', '性别_work', '最高学历_work', 'o_id_work'
])

final_merged_df

In [ ]:
# 将 final_merged_df 保存为 CSV 文件
output_path = r"D:\songyue\tongtong\ORCID数据检查\final_merged_df.csv"
final_merged_df.to_csv(output_path, index=False)

In [ ]:
import pandas as pd
input_path = r"D:\songyue\tongtong\ORCID数据检查\final_merged_df.csv"
final_merged_df = pd.read_csv(input_path)

In [ ]:
final_merged_df

In [ ]:
print(final_merged_df.columns)

博士当前工作值为0,表明学业就业跨国；博士当前工作值为1，表明学业就业不跨国。

In [ ]:
import numpy as np

# 添加“博士当前工作”列，判断“国家唯一ID_edu”是否等于“国家唯一ID_work”
final_merged_df['博士当前工作'] = np.where(final_merged_df['国家唯一ID_edu'] == final_merged_df['国家唯一ID_work'], 1, 0)

# 输出结果 DataFrame
print(final_merged_df[['国家唯一ID_edu', '国家唯一ID_work', '博士当前工作']].head())


In [ ]:
print(final_merged_df.shape)

计算基尼系数：当前工作单位在大学、industry、gov、labs;最高学历分别为博士、硕士、本科、不详

当前工作单位在大学

In [ ]:
import pandas as pd

# 假设 final_merged_df 已经定义并包含“教育类型”、“国家唯一ID_work”、“唯一ID_edu”等列

# 限制 type_work 取值为 "university" 且 "博士当前工作" 列为 1
filtered_df = final_merged_df[(final_merged_df['type_work'] == 'university') & (final_merged_df['博士当前工作'] == 1)]

# 定义用于存储结果的列表
results = []

# 根据“教育类型”分组
for edu_type, group in filtered_df.groupby('教育类型'):
    # 统计国家唯一ID_work频率，并取前3个国家
    top_countries = group['国家唯一ID_work'].value_counts().head(100).index.tolist()
    
    # 遍历前3个国家，获取每个国家的数据
    for country in top_countries:
        # 筛选出属于该国家的样本
        country_samples = group[group['国家唯一ID_work'] == country]
        
        # 统计唯一ID_edu的频率
        id_counts = country_samples['唯一ID_edu'].value_counts().reset_index()
        id_counts.columns = ['唯一ID_edu', '计数']
        
        # 去掉计数小于3的记录/5
        id_counts = id_counts[id_counts['计数'] >= 6]
        
        # 计算剩余记录的总计数
        valid_total_count = id_counts['计数'].sum()
        
        # 如果没有满足条件的记录，则跳过
        if valid_total_count == 0:
            continue
        
        # 重新计算占比
        id_counts['占比'] = id_counts['计数'] / valid_total_count
        
        # 添加 o_id 列（每行中的唯一ID_edu值）
        id_counts['o_id'] = id_counts['唯一ID_edu']
        
        # 添加教育类型和国家字段
        id_counts['教育类型'] = edu_type
        id_counts['国家'] = country
        
        # 将结果添加到列表中
        results.append(id_counts)

# 将所有结果合并为一个 DataFrame
final_result_df = pd.concat(results, ignore_index=True)

# 输出结果表格
print(final_result_df)


In [ ]:
print(final_merged_df.columns)

In [ ]:
# 将 final_merged_df 保存为 CSV 文件
output_path = r"D:\songyue\tongtong\ORCID数据检查\基尼系数计算准备df--大学2.csv"
final_result_df.to_csv(output_path, index=False)

当前工作单位在企业会怎么样？

当前工作单位在企业

In [ ]:
import pandas as pd

# 假设 final_merged_df 已经定义并包含“教育类型”、“国家唯一ID_work”、“唯一ID_edu”等列

# 限制 type_work 取值为 "university" 且 "博士当前工作" 列为 1
filtered_df = final_merged_df[(final_merged_df['type_work'] == 'university') & (final_merged_df['博士当前工作'] == 1)]

# 定义用于存储结果的列表
results = []

# 根据“教育类型”分组
for edu_type, group in filtered_df.groupby('教育类型'):
    # 统计国家唯一ID_work频率，并取前3个国家
    top_countries = group['国家唯一ID_work'].value_counts().head(100).index.tolist()
    
    # 遍历前3个国家，获取每个国家的数据
    for country in top_countries:
        # 筛选出属于该国家的样本
        country_samples = group[group['国家唯一ID_work'] == country]
        
        # 统计唯一ID_edu的频率
        id_counts = country_samples['唯一ID_edu'].value_counts().reset_index()
        id_counts.columns = ['唯一ID_edu', '计数']
        
        # 去掉计数小于3的记录/5
        id_counts = id_counts[id_counts['计数'] >= 3]
        
        # 计算剩余记录的总计数
        valid_total_count = id_counts['计数'].sum()
        
        # 如果没有满足条件的记录，则跳过
        if valid_total_count == 0:
            continue
        
        # 重新计算占比
        id_counts['占比'] = id_counts['计数'] / valid_total_count
        
        # 添加 o_id 列（每行中的唯一ID_edu值）
        id_counts['o_id'] = id_counts['唯一ID_edu']
        
        # 添加教育类型和国家字段
        id_counts['教育类型'] = edu_type
        id_counts['国家'] = country
        
        # 将结果添加到列表中
        results.append(id_counts)

# 将所有结果合并为一个 DataFrame
final_result_df = pd.concat(results, ignore_index=True)

# 将 final_merged_df 保存为 CSV 文件
output_path = r"D:\songyue\tongtong\ORCID数据检查\基尼系数计算准备df--大学1.csv"
final_result_df.to_csv(output_path, index=False)

In [ ]:
# 假设 final_merged_df 已定义，查看 'type_work' 列的唯一取值
unique_type_work_values = final_merged_df['type_work'].unique()
unique_type_work_values

合并不同的类型

In [ ]:
import pandas as pd
import glob

# 定义文件路径模式
file_paths = glob.glob(r'D:\songyue\tongtong\ORCID数据检查\不同类型工作-不同学历——院校分布\二八结果2-*.csv')

# 检查找到的文件
if not file_paths:
    print("未找到任何匹配的文件，请检查文件路径和文件名模式。")
else:
    print("找到以下文件:", file_paths)

    # 用于存储每个文件的数据框列表
    dfs = []

    # 逐个读取文件并添加 '表格名' 列
    for file_path in file_paths:
        # 读取文件
        df = pd.read_csv(file_path)
        
        # 提取文件名作为新的列值
        table_name = file_path.split('/')[-1].replace("二八结果2-", "").replace(".csv", "")
        df['表格名'] = table_name  # 添加新的列 '表格名'，值为文件名（不包含扩展名）
        
        # 将当前数据框添加到列表中
        dfs.append(df)

    # 合并所有数据框
    merged_dfy = pd.concat(dfs, ignore_index=True)

    # 显示合并结果的前几行
    print(merged_dfy.head())


In [ ]:
import pandas as pd
import glob

# 定义文件路径模式（假设这些文件都在同一文件夹中，且名称中包含 "二八结果2-"）
file_paths = glob.glob(r'D:\songyue\tongtong\ORCID数据检查\不同类型工作-不同学历——院校分布\二八结果2-*.csv')

# 读取每个文件并根据文件名添加后缀
dfs = {}
for file_path in file_paths:
    # 提取文件名中的类型（如“企业”、“政府”等）
    file_type = file_path.split('/')[-1].replace("二八结果2-", "").replace(".csv", "")
    suffix = f"_{file_type}"  # 添加后缀
    
    # 读取文件
    df = pd.read_csv(file_path)
    
    # 将文件数据存入字典，键为文件后缀
    dfs[suffix] = df

# 逐步将数据框横向合并
# 假设合并的主键为 "教育类型" 和 "国家"，且每个文件都包含这些列
merged_df = dfs.popitem()[1]  # 初始化合并数据框为第一个读取的文件
for suffix, df in dfs.items():
    merged_df = pd.merge(merged_df, df, on=["教育类型", "国家"],  how="outer", suffixes=("", suffix))

# 查看合并后的结果
merged_df.head()


In [ ]:
merged_dfy

In [ ]:
# 将 final_merged_df 保存为 CSV 文件
output_path = r"D:\songyue\tongtong\ORCID数据检查\不同类型工作-不同学历——院校分布\二八结果2--合并1.csv"
merged_df.to_csv(output_path, index=False)

当前工作单位在大学

计算二八法则

In [ ]:
import pandas as pd

# 假设df是包含数据的DataFrame
# 按教育类型和国家分组
grouped = final_result_df.groupby(['教育类型', '国家'])

# 定义用于存储结果的列表
results = []

# 遍历每个组
for (edu_type, country), group in grouped:
    # 按占比降序排列
    group = group.sort_values('占比', ascending=False)
    # 计算累计占比
    group['累计占比'] = group['占比'].cumsum()
    # 找到累计占比刚好超过0.8的行
    threshold_index = group[group['累计占比'] > 0.2].index[0]
    # 统计唯一ID_edu数量
    unique_id_count = group.loc[:threshold_index, '唯一ID_edu'].nunique()
    # 计算该组总的唯一ID_edu个数
    total_unique_id_count = group['唯一ID_edu'].nunique()
    # 计算百分比
    percentage = unique_id_count / total_unique_id_count * 100
    # 将结果添加到列表
    results.append({
        '教育类型': edu_type,
        '国家': country,
        '累计占比超过0.8的唯一ID_edu个数': unique_id_count,
        '占总唯一ID_edu个数的百分比': percentage
    })

# 转换为DataFrame
result_df = pd.DataFrame(results)

# 将 final_merged_df 保存为 CSV 文件
output_path = r"D:\songyue\tongtong\ORCID数据检查\20%的结果2-大学.csv"
result_df.to_csv(output_path, index=False)

In [ ]:
import pandas as pd

# 假设 final_merged_df 已定义

# 第一步：筛选出 "教育类型" 为 "university" 且 "博士当前工作" 为 1 的记录
df1 = final_merged_df[(final_merged_df['教育类型'] == "university") & (final_merged_df['博士当前工作'] == 1)]
print("筛选后的记录数:", len(df1))  # 检查筛选后的数据量

# 第二步：按 "国家唯一ID_work" 和 "教育类型" 分组，统计每组中 "唯一ID_work" 和 "唯一ID_edu" 组合的频率，并计算占比
results = []
for (country_id, edu_type), group in df1.groupby(['国家唯一ID_work', '教育类型']):
    if group.empty:
        print(f"No data for group: 国家唯一ID_work={country_id}, 教育类型={edu_type}")
    else:
        # 统计 (唯一ID_work, 唯一ID_edu) 组合的频率
        combo_counts = group.groupby(['唯一ID_work', '唯一ID_edu']).size().reset_index(name='计数')
        
        # 计算每组的总计数
        total_count = combo_counts['计数'].sum()
        
        # 计算占比
        combo_counts['占比'] = combo_counts['计数'] / total_count
        
        # 添加 国家唯一ID_work 和 教育类型 列
        combo_counts['国家唯一ID_work'] = country_id
        combo_counts['教育类型'] = edu_type
        
        # 将结果添加到列表中
        results.append(combo_counts)

# 检查 results 列表是否为空
if results:
    # 将所有分组结果合并为一个 DataFrame
    final_result_df = pd.concat(results, ignore_index=True)
    print("合并结果的前几行:\n", final_result_df.head())
else:
    print("No data to concatenate in results.")

In [ ]:
df1

基尼系数计算

In [ ]:
import pandas as pd

# 假设 data 是包含数据的原始 DataFrame
# 按“教育类型”和“国家”组合进行分组
grouped = final_result_df.groupby(['教育类型', '国家'])

# 定义Excel写入路径
output_path = r'D:\songyue\tongtong\ORCID数据检查\分组基尼系数计算2.xlsx'

# 使用ExcelWriter创建一个多Sheet的Excel文件
with pd.ExcelWriter(output_path) as writer:
    # 遍历每个子 DataFrame
    for (edu_type, country), df in grouped:
        # 检查子数据框是否为空
        if df.empty:
            continue

        # 计算 x 占比和 x 累计占比
        total_rows = len(df['唯一ID_edu'])
        df['x占比'] = 1 / total_rows
        df['x累计占比'] = df['x占比'].cumsum()
        
        # 计算 y 占比和 y 累计占比
        total_count = df['计数'].sum()
        if total_count == 0:
            continue
        df['y占比'] = df['计数'] / total_count
        df['y累计占比'] = df['y占比'].cumsum()
        
        # 计算梯形面积
        df['梯形面积'] = (df['x累计占比'].diff() * (df['y累计占比'] + df['y累计占比'].shift(fill_value=0))) / 2
        df.at[0, '梯形面积'] = (df['x累计占比'].iloc[0] * df['y累计占比'].iloc[0]) / 2
        
        # 重置索引，确保索引从0开始
        df = df.reset_index(drop=True)

        # 计算梯形面积之和并赋值到第一行
        df.at[0, '梯形面积之和'] = df['梯形面积'].sum()

        # 计算基尼系数并赋值到第一行
        df.at[0, '基尼系数'] = 2 * df.at[0, '梯形面积之和'] - 1

    
        # 将当前子DataFrame写入到Excel中的一个Sheet
        sheet_name = f"{edu_type}_{int(country)}"  # 创建Sheet名称
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"结果已保存到: {output_path}")

In [ ]:
import pandas as pd

# 假设 data 是包含数据的原始 DataFrame
# 按“教育类型”和“国家”组合进行分组
grouped = final_result_df.groupby(['教育类型', '国家'])

# 定义用于存储每个子df基尼系数的列表
gini_results = []

# 遍历每个子 DataFrame
for (edu_type, country), df in grouped:
    # 检查子数据框是否为空
    if df.empty:
        continue

    # 计算 x 占比和 x 累计占比
    total_rows = len(df['唯一ID_edu'])
    df['x占比'] = 1 / total_rows
    df['x累计占比'] = df['x占比'].cumsum()
    
    # 计算 y 占比和 y 累计占比
    total_count = df['计数'].sum()
    if total_count == 0:
        # 如果总计数为0，则跳过该分组
        continue
    df['y占比'] = df['计数'] / total_count
    df['y累计占比'] = df['y占比'].cumsum()
    
    # 计算梯形面积
    df['梯形面积'] = (df['x累计占比'].diff() * (df['y累计占比'] + df['y累计占比'].shift(fill_value=0))) / 2
    df.at[0, '梯形面积'] = (df['x累计占比'].iloc[0] * df['y累计占比'].iloc[0]) / 2
    
    # 重置索引，确保索引从0开始
    df = df.reset_index(drop=True)

    # 计算梯形面积之和并赋值到第一行
    df.at[0, '梯形面积之和'] = df['梯形面积'].sum()

    # 计算基尼系数并赋值到第一行
    df.at[0, '基尼系数'] = 2 * df.at[0, '梯形面积之和'] - 1
    
    # 提取当前子 DataFrame 的基尼系数的第一个值
    gini_value = df['基尼系数'].iloc[0]
    # 检查基尼系数是否为 NaN
    if pd.notna(gini_value):
        gini_results.append({'教育类型': edu_type, '国家': country, '基尼系数': gini_value})

# 将结果转换为 DataFrame
gini_result_df = pd.DataFrame(gini_results)

gini_result_df
# 将 final_merged_df 保存为 CSV 文件
output_path = r"D:\songyue\tongtong\ORCID数据检查\基尼系数汇总2.csv"
gini_result_df.to_csv(output_path, index=False)

In [ ]:
import pandas as pd

# 假设 final_merged_df 已经定义并包含“教育类型”、“国家唯一ID_work”、“唯一ID_edu”等列

# 限制 type_work 取值为 "university" 且 "博士当前工作" 列为 1
filtered_df = final_merged_df[(final_merged_df['type_work'] == 'university') & (final_merged_df['博士当前工作'] == 0)]

# 定义用于存储结果的列表
results = []

# 根据“教育类型”分组
for edu_type, group in filtered_df.groupby('教育类型'):
    # 统计国家唯一ID_work频率，并取前3个国家
    top_countries = group['国家唯一ID_work'].value_counts().head(100).index.tolist()
    
    # 遍历前3个国家，获取每个国家的数据
    for country in top_countries:
        # 筛选出属于该国家的样本
        country_samples = group[group['国家唯一ID_work'] == country]
        
        # 统计唯一ID_edu的频率
        id_counts = country_samples['国家唯一ID_edu'].value_counts().reset_index()
        id_counts.columns = ['国家唯一ID_edu', '计数']
        
        # 去掉计数小于3的记录
        id_counts = id_counts[id_counts['计数'] >= 4]
        
        # 计算剩余记录的总计数
        valid_total_count = id_counts['计数'].sum()
        
        # 如果没有满足条件的记录，则跳过
        if valid_total_count == 0:
            continue
        
        # 重新计算占比
        id_counts['占比'] = id_counts['计数'] / valid_total_count
        
        # 添加 o_id 列（每行中的唯一ID_edu值）
        id_counts['o_id'] = id_counts['国家唯一ID_edu']
        
        # 添加教育类型和国家字段
        id_counts['教育类型'] = edu_type
        id_counts['国家'] = country
        
        # 将结果添加到列表中
        results.append(id_counts)

# 将所有结果合并为一个 DataFrame
final_result_df = pd.concat(results, ignore_index=True)

# 输出结果表格
print(final_result_df)


In [ ]:
# 将 final_merged_df 保存为 CSV 文件
output_path = r"D:\songyue\tongtong\ORCID数据检查\基尼系数计算准备df--海归来源国.csv"
final_result_df.to_csv(output_path, index=False)

In [1]:
import pandas as pd

# 假设 final_merged_df 已经定义并包含“教育类型”、“国家唯一ID_work”、“唯一ID_edu”等列

# 限制 type_work 取值为 "university" 且 "博士当前工作" 列为 1
filtered_df = final_merged_df[(final_merged_df['type_work'] == 'university') & (final_merged_df['博士当前工作'] == 0)]

# 定义用于存储结果的列表
results = []

# 根据“教育类型”分组
for edu_type, group in filtered_df.groupby('教育类型'):
    # 统计国家唯一ID_work频率，并取前3个国家
    top_countries = group['国家唯一ID_work'].value_counts().head(100).index.tolist()
    
    # 遍历前3个国家，获取每个国家的数据
    for country in top_countries:
        # 筛选出属于该国家的样本
        country_samples = group[group['国家唯一ID_work'] == country]
        
        # 统计唯一ID_edu的频率
        id_counts = country_samples['唯一ID_edu'].value_counts().reset_index()
        id_counts.columns = ['唯一ID_edu', '计数']
        
        # 去掉计数小于3的记录
        # id_counts = id_counts[id_counts['计数'] >= 4]
        
        # 计算剩余记录的总计数
        valid_total_count = id_counts['计数'].sum()
        
        # 如果没有满足条件的记录，则跳过
        if valid_total_count == 0:
            continue
        
        # 重新计算占比
        id_counts['占比'] = id_counts['计数'] / valid_total_count
        
        # 添加 o_id 列（每行中的唯一ID_edu值）
        id_counts['o_id'] = id_counts['唯一ID_edu']
        
        # 添加教育类型和国家字段
        id_counts['教育类型'] = edu_type
        id_counts['国家'] = country
        
        # 将结果添加到列表中
        results.append(id_counts)

# 将所有结果合并为一个 DataFrame
final_result_df = pd.concat(results, ignore_index=True)

# 输出结果表格
print(final_result_df)


NameError: name 'final_merged_df' is not defined

In [ ]:
# 将 final_merged_df 保存为 CSV 文件
output_path = r"D:\songyue\tongtong\ORCID数据检查\基尼系数计算准备df--海归来源校.csv"
final_result_df.to_csv(output_path, index=False)

In [ ]:
import pandas as pd

# 假设 final_merged_df 已定义

# 第一步：筛选出 "教育类型" 为 "university" 且 "博士当前工作" 为 1 的记录
df1 = final_merged_df[(final_merged_df['教育类型'] == "university") & (final_merged_df['博士当前工作'] == 1)]

# 按 "国家唯一ID_work"、"教育类型" 和 "唯一ID_work" 分组，统计每组中 "唯一ID_edu" 的频率，并计算占比
results = []
for (country_id, edu_type, unique_work_id), group in df1.groupby(['国家唯一ID_work', '教育类型', '唯一ID_work']):
    # 统计唯一ID_edu的频率
    id_counts = group['唯一ID_edu'].value_counts().reset_index()
    id_counts.columns = ['唯一ID_edu', '计数']
    
    # 计算总计数
    total_count = id_counts['计数'].sum()
    
    # 计算占比
    id_counts['占比'] = id_counts['计数'] / total_count
    
    # 添加国家唯一ID_work、教育类型 和 唯一ID_work 列
    id_counts['国家唯一ID_work'] = country_id
    id_counts['教育类型'] = edu_type
    id_counts['唯一ID_work'] = unique_work_id
    
    # 将结果添加到列表中
    results.append(id_counts)

# 将所有分组结果合并为一个 DataFrame
final_result_df = pd.concat(results, ignore_index=True)

# 查看结果
final_result_df.head()



区分样本：分析有工作的样本——当前工作机构与本科院校国家的关系

区分样本：分析有工作，且当前工作在大学

区分样本：分析有工作，且当前工作在企业

In [ ]:
# 查看 '工作统计新' 和 '教育统计新' 列的唯一取值
unique_values_work = merged_df_new3['工作统计新'].unique()
unique_values_edu = merged_df_new3['教育统计新'].unique()

# 打印唯一值列表
print("工作统计新的唯一值:", unique_values_work)
print("教育统计新的唯一值:", unique_values_edu)

# 统计唯一值的个数
unique_count_work = merged_df_new3['工作统计新'].nunique()
unique_count_edu = merged_df_new3['教育统计新'].nunique()

# 打印唯一值的数量
print("工作统计新的唯一值数量:", unique_count_work)
print("教育统计新的唯一值数量:", unique_count_edu)

In [ ]:
# 统计 '工作统计新' 和 '教育统计新' 列的组合频率
combination_frequency = merged_df_new3[['工作统计新', '教育统计新']].value_counts().reset_index()
# 重命名列
combination_frequency.columns = ['工作统计新', '教育统计新', '频率']
# 打印结果
combination_frequency

In [ ]:
# 筛选出 '工作统计新' 列等于1的行
sub_df1 = merged_df_new3[merged_df_new3['工作统计新'] == 1]

# 统计 'o_id' 列的唯一取值及频率
o_id_counts1 = sub_df1['o_id'].value_counts().reset_index()
o_id_counts1.columns = ['o_id', '频率']
o_id_counts1['类型'] = '工作统计'

# 统计 '国家唯一ID' 列的唯一取值及频率
unique_id_counts1 = sub_df1['国家唯一ID'].value_counts().reset_index()
unique_id_counts1.columns = ['国家唯一ID', '频率']
unique_id_counts1['类型'] = '工作统计'

# 统计 '所在洲' 列的唯一取值及频率
continent_counts1 = sub_df1['所在洲'].value_counts().reset_index()
continent_counts1.columns = ['所在洲', '频率']
continent_counts1['类型'] = '工作统计'

# 筛选出 '教育统计新' 列等于1的行
sub_df2 = merged_df_new3[merged_df_new3['教育统计新'] == 1]

# 统计 'o_id' 列的唯一取值及频率
o_id_counts2 = sub_df2['o_id'].value_counts().reset_index()
o_id_counts2.columns = ['o_id', '频率']
o_id_counts2['类型'] = '教育统计'

# 统计 '国家唯一ID' 列的唯一取值及频率
unique_id_counts2 = sub_df2['国家唯一ID'].value_counts().reset_index()
unique_id_counts2.columns = ['国家唯一ID', '频率']
unique_id_counts2['类型'] = '教育统计'

# 统计 '所在洲' 列的唯一取值及频率
continent_counts2 = sub_df2['所在洲'].value_counts().reset_index()
continent_counts2.columns = ['所在洲', '频率']
continent_counts2['类型'] = '教育统计'



In [ ]:
# 合并所有结果
final_result = pd.concat([unique_id_counts1, continent_counts1,
                           unique_id_counts2, continent_counts2], 
                          ignore_index=True)

final_result
final_result.to_csv(r"D:\songyue\tongtong\ORCID数据检查\filtered_check_统计.csv", index=False)

In [ ]:
import pandas as pd

# 定义一个通用函数来统计不同类型数据的唯一值数量
def calculate_unique_counts(df, filter_column, filter_value, group_column, count_column, type_label):
    # 筛选出特定条件的行
    filtered_df = df[(df[filter_column] == filter_value) & (df[group_column].notna())]
    
    # 统计每个唯一取值对应的 count_column 唯一值数量
    count_result = filtered_df.groupby(group_column)[count_column].nunique().reset_index()
    count_result.columns = [group_column, f'{count_column}数量']
    count_result['类型'] = type_label
    
    return count_result

# 应用函数统计数据并合并结果
# 统计 '国家唯一ID' 列和 '所在洲' 列的唯一取值
o_id_counts_country_work = calculate_unique_counts(merged_df_new3, '工作统计新', 1, '国家唯一ID', 'o_id', '工作统计')
o_id_counts_country_education = calculate_unique_counts(merged_df_new3, '教育统计新', 1, '国家唯一ID', 'o_id', '教育统计')

o_id_counts_continent_work = calculate_unique_counts(merged_df_new3, '工作统计新', 1, '所在洲', 'o_id', '工作统计')
o_id_counts_continent_education = calculate_unique_counts(merged_df_new3, '教育统计新', 1, '所在洲', 'o_id', '教育统计')

# 合并所有结果
final_result_country = pd.concat([o_id_counts_country_work, o_id_counts_country_education], ignore_index=True)
final_result_continent = pd.concat([o_id_counts_continent_work, o_id_counts_continent_education], ignore_index=True)

# 保存结果到CSV文件
final_result_country.to_csv(r"D:\songyue\tongtong\ORCID数据检查\filtered_check_国家唯一ID_统计.csv", index=False)
final_result_continent.to_csv(r"D:\songyue\tongtong\ORCID数据检查\filtered_check_所在洲_统计.csv", index=False)

# 输出合并结果
print("\n国家唯一ID统计结果：")
print(final_result_country)
print("\n所在洲统计结果：")
print(final_result_continent)


In [ ]:
# 筛选 '频率' 大于 50 的行
o_id_counts_filtered = o_id_counts[o_id_counts['频率'] > 50]

# 打印结果
print("频率大于50的 o_id 及其频率：")
print(o_id_counts_filtered)

In [ ]:
# 筛选出频率大于 50 的 o_id
o_id_high_frequency = o_id_counts[o_id_counts['频率'] > 50]['o_id']

# 从 merged_df_new2 中筛选出 o_id 在 o_id_high_frequency 中的行
filtered_df = merged_df_new2[merged_df_new2['o_id'].isin(o_id_high_frequency)]
# 将 filtered_df 导出为 CSV 文件
filtered_df.to_csv(r"D:\songyue\tongtong\ORCID数据检查\filtered_check_work.csv", index=False)

In [ ]:
import pandas as pd

# 读取三个CSV文件并指定数据框名称
edu = pd.read_csv(r"D:\songyue\tongtong\每个o_id教育经历统计.csv")
work= pd.read_csv(r"D:\songyue\tongtong\每个o_id工作经历统计.csv")
allex = pd.read_csv(r"D:\songyue\tongtong\每个o_id所有经历统计.csv")

# 给每个数据框添加后缀
# edu = edu.add_suffix('_edu')
# work = work.add_suffix('_work')
# allex = allex.add_suffix('_allex')

# 给每个数据框除 'o_id' 列外的其他列添加后缀
edu = edu.add_suffix('_edu').rename(columns={'o_id_edu': 'o_id'})
work = work.add_suffix('_work').rename(columns={'o_id_work': 'o_id'})
allex = allex.add_suffix('_allex').rename(columns={'o_id_allex': 'o_id'})

# 基于 'o_id' 列进行合并
merged_df = edu.merge(work, on='o_id', how='outer').merge(allex, on='o_id', how='outer')

# 显示合并后的数据框
merged_df

# # 合并数据框（假设按行索引合并）
# merged_df = pd.concat([df1, df2, df3], axis=1)

In [ ]:
print(merged_df.columns)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# 设置字体以支持中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']  # 使用黑体（SimHei）字体
plt.rcParams['axes.unicode_minus'] = False    # 解决坐标轴负号显示问题

# 假设 merged_df 是包含这些列的数据框
# 如果还没有读取数据，请在此处将合并后的CSV文件导入为 merged_df
# merged_df = pd.read_csv("merged_file.csv")

# 选择需要分析的列
columns_to_analyze = [
    '国家唯一ID个数_edu', '所在洲个数_edu', '城市个数_edu', 'type个数_edu', '教育统计新求和_edu', 
    '国家唯一ID个数_work', '所在洲个数_work', '城市个数_work', 'type个数_work', '工作统计新求和_work', 
    '国家唯一ID个数_allex', '所在洲个数_allex', '城市个数_allex', 'type个数_allex', 
    '教育统计新求和_allex', '工作统计新求和_allex'
]
# 取出这些列并计算相关系数矩阵
correlation_matrix = merged_df[columns_to_analyze].corr()

# 设置热图的尺寸
plt.figure(figsize=(12, 10))
# 绘制热图
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap="coolwarm", linewidths=0.5)
plt.title("Correlation Matrix of Selected Columns")
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import zscore
from scipy.cluster.hierarchy import linkage, leaves_list

# 示例数据框（使用你实际的 merged_df）
# merged_df = pd.read_csv("merged_file.csv")

# 选择需要分析的列
columns_to_analyze = [
    '国家唯一ID个数_edu', '所在洲个数_edu', '城市个数_edu', 'type个数_edu', '教育统计新求和_edu', 
    '国家唯一ID个数_work', '所在洲个数_work', '城市个数_work', 'type个数_work', '工作统计新求和_work', 
    '国家唯一ID个数_allex', '所在洲个数_allex', '城市个数_allex', 'type个数_allex', 
    '教育统计新求和_allex', '工作统计新求和_allex'
]

# 计算相关系数矩阵
correlation_matrix = merged_df[columns_to_analyze].corr()

# 使用层次聚类对相关系数矩阵进行重排
linkage_matrix = linkage(correlation_matrix, method='average')
ordered_index = leaves_list(linkage_matrix)

# 根据聚类顺序重新排列矩阵
ordered_corr_matrix = correlation_matrix.iloc[ordered_index, ordered_index]

# 绘制重新排序的热图x
plt.figure(figsize=(12, 10))
ax = sns.heatmap(
    ordered_corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", linewidths=0.5, 
    cbar_kws={"orientation": "horizontal"}
)
# plt.title("Reordered Correlation Matrix (After Clustering)", y=1.1)  # 调整标题位置
ax.xaxis.set_label_position('top')  # 设置标签位置
ax.xaxis.tick_top()  # 将x轴的刻度移到顶部
plt.xticks(rotation=45, ha="left")  # 将横坐标标签倾斜
# 保存图片到指定路径
output_path = r"D:\songyue\tongtong\地理位置reordered_correlation_matrix.png"
plt.savefig(output_path, dpi=300, bbox_inches="tight")
plt.show()